In [1]:
import abipy
import numpy as np
import math 
import cmath
import abipy
from abipy.electrons.scr import ScrFile
import time
import matplotlib.pyplot as plt
import pointcloud as pc
from scipy.interpolate import RegularGridInterpolator
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
import Fourier_tool as Ft
import XGChi0 
import MatrixCharacteristic
from abipy import abilab
import abipy.data as abidata
from abipy.abilab import Structure
import random
from abipy.abilab import abiopen
import abipy.data as abidata

In [2]:
filename = "BS_Bulk2Atoms_GSR.nc"

In [3]:
with abiopen(filename) as ncfile:
    ebands = ncfile.ebands
ebands.plotly(with_gaps=False)

# Test Symmetries

In [4]:
filename = "Bulk2Atoms6_ecut8o_DS1_SUS.nc"
#BUILD
chi0GG, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk = XGChi0.Build_Chi0GG(filename, "FromSym", omega = 0)

Opening the file Bulk2Atoms6_ecut8o_DS1_SUS.nc containing a matrix chi^0[q, G, G'] with  40 q points in the IBZ and  229 G vectors
The algorithm will use  32 symmetries to build the matrix chi^0[q, G, G'] with q in the BZ
the initialization of the dictionnaries and gathering of the information about the valid vectors took 3.412770986557007 seconds
The initialisation of smallchi0GG took 462.6366171836853 seconds
4.337012082800129e-09
4.337012082800129e-09
3.893263275244092e-15
3.893263275244092e-15
the building of chi0GG took  82.89155769348145 seconds


In [ ]:
smallchi0_test = XGChi0.Sym_chi0GG("Bulk2Atoms6_ecut8o_DS1_SUS.nc", omega = 0)

In [5]:
qG_vec_to_ind = {}
for i in ind_qG_to_vec.keys():
    qG = ind_qG_to_vec[i]
    qG_vec_to_ind[qG] = i

qvec_to_ind = {}
for i in ind_q_to_vec.keys():
    q = ind_q_to_vec[i]
    qvec_to_ind[q] = i

Gvec_to_ind = {}
for i in ind_G_to_vec.keys():
    Gvec = ind_G_to_vec[i]
    Gvec_to_ind[Gvec] = i

In [33]:
print(ind_q_to_vec)

{0: (0.0, 0.0, 0.0), 1: (1.0, 0.0, 0.0), 2: (2.0, 0.0, 0.0), 3: (3.0, 0.0, 0.0), 4: (1.0, 1.0, 0.0), 5: (2.0, 1.0, 0.0), 6: (3.0, 1.0, 0.0), 7: (2.0, 2.0, 0.0), 8: (3.0, 2.0, 0.0), 9: (3.0, 3.0, 0.0), 10: (0.0, 0.0, 1.0), 11: (1.0, 0.0, 1.0), 12: (2.0, 0.0, 1.0), 13: (3.0, 0.0, 1.0), 14: (1.0, 1.0, 1.0), 15: (2.0, 1.0, 1.0), 16: (3.0, 1.0, 1.0), 17: (2.0, 2.0, 1.0), 18: (3.0, 2.0, 1.0), 19: (3.0, 3.0, 1.0), 20: (0.0, 0.0, 2.0), 21: (1.0, 0.0, 2.0), 22: (2.0, 0.0, 2.0), 23: (3.0, 0.0, 2.0), 24: (1.0, 1.0, 2.0), 25: (2.0, 1.0, 2.0), 26: (3.0, 1.0, 2.0), 27: (2.0, 2.0, 2.0), 28: (3.0, 2.0, 2.0), 29: (3.0, 3.0, 2.0), 30: (0.0, 0.0, 3.0), 31: (1.0, 0.0, 3.0), 32: (2.0, 0.0, 3.0), 33: (3.0, 0.0, 3.0), 34: (1.0, 1.0, 3.0), 35: (2.0, 1.0, 3.0), 36: (3.0, 1.0, 3.0), 37: (2.0, 2.0, 3.0), 38: (3.0, 2.0, 3.0), 39: (3.0, 3.0, 3.0), 40: (-1.0, 0.0, 0.0), 41: (-2.0, 0.0, 0.0), 42: (-1.0, -1.0, 0.0), 43: (-2.0, -1.0, 0.0), 44: (3.0, -1.0, 0.0), 45: (-2.0, -2.0, 0.0), 46: (3.0, -2.0, 0.0), 47: (0.0, 0.

In [6]:
structure_prim=abipy.core.structure.Structure.from_file("Bulk2Atoms6_ecut8o_DS1_SUS.nc")
structure_prim

Structure Summary
Lattice
    abc : 2.8535866547158504 2.8535866547158504 4.035580948506026
 angles : 90.0 90.0 90.0
 volume : 32.86156131033388
      A : 2.017790474253013 -2.017790474253013 0.0
      B : 2.017790474253013 2.017790474253013 0.0
      C : 0.0 0.0 4.035580948506026
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (2.0178, 0.0000, 2.0178) [0.5000, 0.5000, 0.5000]

In [7]:
i, j, k = 3, 6, 7
q = ind_q_to_vec[i]
G1 = ind_G_to_vec[j]
G2 = ind_G_to_vec[k]
qvec = [q[0], q[1], q[2]]
G1vec = [G1[0], G1[1], G1[2]]
G2vec = [G2[0], G2[1], G2[2]]
Sym = abipy.core.symmetries.AbinitSpaceGroup.from_structure(structure_prim)
SymR = Sym.symrec
nsym = len(SymR)
print(nsym)
count = 0
for i in range(nsym):
    S = SymR[i]
    Sq = np.matmul(S, qvec)
    SG1 = np.matmul(S, G1vec)
    SG2 = np.matmul(S, G2vec)
    SqG1 = Sq + SG1
    SqG2 = Sq + SG2
    print(Sq, SG1, SG2)
    if (SqG1[0], SqG1[1], SqG1[2]) in qG_vec_to_ind.keys() and (SqG2[0], SqG2[1], SqG2[2]) in qG_vec_to_ind.keys():
        count += 1
        if np.amax(Sq) > 3 or np.amin(Sq) <= -3:
            print(np.amax(Sq) > 3)
            print(np.amin(Sq) <= -3)
            a, b, c = Sq[0], Sq[1], Sq[2]
            if Sq[0]>3:
                a=round(Sq[0]-6,3)
            elif Sq[0]<=-3:
                a=round(Sq[0]+6,3)
            if Sq[1]>3:
                b=round(Sq[1]-6,3)
            elif Sq[1]<=-3:
                b=round(Sq[1]+6,3)
            if Sq[2]>3:
                c=round(Sq[2]-6,3)
            elif Sq[2]<=-3:
                c=round(Sq[2]+6,3)
            q_in_bz = (a, b, c)
            SG1 = (SqG1[0]-q_in_bz[0], SqG1[1]-q_in_bz[1], SqG1[2]-q_in_bz[2])
            SG2 = (SqG2[0]-q_in_bz[0], SqG2[1]-q_in_bz[1], SqG2[2]-q_in_bz[2])
        else:
            q_in_bz = (Sq[0], Sq[1], Sq[2])
            print(q_in_bz)    
        indq = qvec_to_ind[q_in_bz]
        indG1 = Gvec_to_ind[(SG1[0], SG1[1], SG1[2])]
        indG2 = Gvec_to_ind[(SG2[0], SG2[1], SG2[2])]
        print(chi0GG[indq, indG1, indG2])
print(count==nsym)

32
[3. 0. 0.] [ 0. -6.  0.] [6. 0. 6.]
(3.0, 0.0, 0.0)
(2.5064342094871517e-11+5.5578652791155037e-11j)
[-3.  0.  0.] [0. 6. 0.] [-6.  0. -6.]
False
True
(2.5064342094871517e-11+5.5578652791155037e-11j)
[0. 3. 0.] [6. 0. 0.] [0. 6. 6.]
(0.0, 3.0, 0.0)
(2.5064342094871517e-11+5.5578652791155037e-11j)
[ 0. -3.  0.] [-6.  0.  0.] [ 0. -6. -6.]
False
True
(2.5064342094871517e-11+5.5578652791155037e-11j)
[-3.  0.  0.] [0. 6. 0.] [-6.  0.  6.]
False
True
(2.5064342094871517e-11+5.5578652791155037e-11j)
[3. 0. 0.] [ 0. -6.  0.] [ 6.  0. -6.]
(3.0, 0.0, 0.0)
(2.5064342094871517e-11+5.5578652791155037e-11j)
[ 0. -3.  0.] [-6.  0.  0.] [ 0. -6.  6.]
False
True
(2.5064342094871517e-11+5.5578652791155037e-11j)
[0. 3. 0.] [6. 0. 0.] [ 0.  6. -6.]
(0.0, 3.0, 0.0)
(2.5064342094871517e-11+5.5578652791155037e-11j)
[0. 3. 0.] [-6.  0.  0.] [ 0.  6. -6.]
(0.0, 3.0, 0.0)
(2.5064342094871517e-11+5.5578652791155037e-11j)
[ 0. -3.  0.] [6. 0. 0.] [ 0. -6.  6.]
False
True
(2.5064342094871517e-11+5.55786527911

In [49]:
filename = "Slab5Atoms6_ecut8o_DS1_SUS.nc"
#BUILD
chi0GG_t, ind_qG_to_vec, n1, n2, n3, ind_q_to_vec, ind_G_to_vec, G, nk = XGChi0.Build_Chi0GG(filename, "FromSym", omega = 0)

Opening the file Slab5Atoms6_ecut8o_DS1_SUS.nc containing a matrix chi^0[q, G, G'] with  10 q points in the IBZ and  1221 G vectors
The algorithm will use  16 symmetries to build the matrix chi^0[q, G, G'] with q in the BZ
the initialization of the dictionnaries and gathering of the information about the valid vectors took 2.501829147338867 seconds
The initialisation of smallchi0GG took 913.2014558315277 seconds
1.0763371527389154e-08
1.0763371527389154e-08
4.112530459135077e-15
4.112530459135077e-15
the building of chi0GG took  575.2925229072571 seconds


In [48]:
print(chi0GG_t)

[[[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j -8.60809833e-02-6.06522471e-12j
    1.20867288e-03-1.48595372e-12j ... -1.41418861e-04-2.11629770e-11j
   -1.41418861e-04-2.11629770e-11j  3.81882103e-06+2.30958644e-12j]
  [ 0.00000000e+00+0.00000000e+00j  1.20867288e-03-1.48595372e-12j
   -8.60809833e-02-6.06522471e-12j ...  3.81882103e-06+2.30958644e-12j
    3.81882103e-06+2.30958644e-12j -1.41418861e-04-2.11629770e-11j]
  ...
  [ 0.00000000e+00+0.00000000e+00j -1.41418861e-04+2.11384783e-11j
    3.81882177e-06-2.30151635e-12j ... -5.35925373e-03-4.63192674e-14j
    8.50993797e-06+6.43439024e-14j  6.77793032e-05+2.04480672e-13j]
  [ 0.00000000e+00+0.00000000e+00j -1.41418861e-04+2.11384783e-11j
    3.81882177e-06-2.30151635e-12j ...  8.50993797e-06+6.43439024e-14j
   -5.35925373e-03-4.63192674e-1

In [46]:
print(chi0GG_t)

[[[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j -8.60809833e-02-6.06522471e-12j
    1.20867288e-03-1.48595372e-12j ... -1.41418861e-04-2.11629770e-11j
   -1.41418861e-04-2.11629770e-11j  3.81882103e-06+2.30958644e-12j]
  [ 0.00000000e+00+0.00000000e+00j  1.20867288e-03-1.48595372e-12j
   -8.60809833e-02-6.06522471e-12j ...  3.81882103e-06+2.30958644e-12j
    3.81882103e-06+2.30958644e-12j -1.41418861e-04-2.11629770e-11j]
  ...
  [ 0.00000000e+00+0.00000000e+00j -1.41418861e-04+2.11384783e-11j
    3.81882177e-06-2.30151635e-12j ... -5.35925373e-03-4.63192674e-14j
    8.50993797e-06+6.43439024e-14j  6.77793032e-05+2.04480672e-13j]
  [ 0.00000000e+00+0.00000000e+00j -1.41418861e-04+2.11384783e-11j
    3.81882177e-06-2.30151635e-12j ...  8.50993797e-06+6.43439024e-14j
   -5.35925373e-03-4.63192674e-1

In [9]:
chi0rr = XGChi0.FFT_chi0_sizeadapt(np.real(chi0GG), ind_qG_to_vec, 48, 48, 66, 8, 8, 11, ind_q_to_vec, ind_G_to_vec, G, nk, opt2 = "Kaltak")

8 8 11
Starting first FFT
8 8 11
The first FFT took  30.637984037399292  seconds
Starting second FFT
The second FFT took  149.6725778579712  seconds


In [12]:
Vbulk = 32.86156131033388*6**3
a, b, c = 2.8535866547158504,  2.8535866547158504, 4.035580948506026
chargeb = XGChi0.charge_accul_cut(chi0rr/Vbulk, 2, "xz", 0)

Max charge accumulation =  2.8229595214036913


In [13]:
fig = go.Figure(data = go.Contour(z = np.real(chargeb), x = np.linspace(-c/2, c/2, 12, endpoint = True), y = np.linspace(-a/2, a/2, 9, endpoint=True), colorscale='sunset', line_smoothing=0.85, contours_coloring='heatmap'))

fig.update_layout(
    autosize=False,
    width=800,
    height=570,
    )

fig.show(dpi = 300)

In [14]:
print(chargeb[3, 9])

(-2.458208552782537-8.733310814302958e-17j)


In [17]:
print(chargeb[3, 2])

(-2.4582085527825375-1.4922107884055524e-18j)
